In [ ]:
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from psycopg2.extensions import AsIs
from datetime import datetime 
import os

In [ ]:
now_dt=datetime.now()
newCompanyFile="1-NewAdminCompany.csv"


if os.path.exists(newCompanyFile)==False:
    raise Exception(f"Not found file {newCompanyFile} on this folder")
else:
    print(f"{newCompanyFile} is master file for adding subcompany ,customer-support, datacenter,branch")

In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error

def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)     
    # print(sql)
    # print(params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 




def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()
    

In [ ]:
print("Load new compnay")
dfMasterAdminComp=pd.read_csv(newCompanyFile)
lisCompID=dfMasterAdminComp['id'].tolist()
print(f"total company : {len(lisCompID)}")

sql_admin_comp=f"""  select id,company_name,company_full_name,company_address from app_company where id  in  %(id_param)s  order by id """
df_new_company=list_data(sql_admin_comp,{"id_param":tuple(lisCompID)},get_postgres_conn())

print(df_new_company.info())

# sub comppany

In [ ]:
dfSubComp=pd.DataFrame(columns=['sub_company_name','sub_company_address','head_company_id'])
mapCols=[ ('sub_company_name','company_full_name'),('sub_company_address','company_address'),('head_company_id','id') ]

for colPair in mapCols:
    dfSubComp[colPair[0]]=df_new_company[colPair[1]]

print("dfSubComp")

print("=====================================================")
print("Adding sub company step") 
    
if len( dfSubComp)==len(df_new_company):
   result=add_data_values(dfSubComp,'app_subcompany',get_postgres_conn())
   sql_subcomp="""
    select company_full_name  ,sub_company_name,app_company.id,app_subcompany.head_company_id
    from app_company inner join app_subcompany on app_company.id = app_subcompany.head_company_id
    where head_company_id  in  %(id_param)s  order by id
   """
   dfNewSubcompany=list_data(sql_subcomp,{"id_param":tuple(lisCompID)},get_postgres_conn()) 
   print(dfNewSubcompany)
   dfNewSubcompany.to_csv('2-SubCompany.csv',index=False)
   print("Ste#1 Added SubCompany Successfully")
print("=====================================================")

# Customer Support

In [ ]:
# Take this dataframe to add branch and datacenter
dfCustSup=pd.DataFrame(columns=['company_id'])
mapCols=[ ('company_id','id')]

for colPair in mapCols:
    dfCustSup[colPair[0]]=df_new_company[colPair[1]]
 
dfCustSup['customer_name']='-'
dfCustSup['customer_telephone']='-'
dfCustSup['customer_email']='-'
dfCustSup['is_active']=True
dfCustSup['updated_at']=now_dt
print(dfCustSup)

print("=====================================================")
print("Adding customer support step")    
if len(dfCustSup)==len(df_new_company):
   result=add_data_values(dfCustSup,'app_customer',get_postgres_conn())
   sql_subcomp="""
select app_customer.id as id,app_customer.customer_name ,
ac.company_name,ac.company_address,app_customer.company_id  from app_customer
inner join app_company ac on ac.id = app_customer.company_id
where app_customer.company_id  in  %(id_param)s  order by id
   """
   dfCustSup=list_data(sql_subcomp,{"id_param":tuple(lisCompID)},get_postgres_conn()) 
   print(dfCustSup)
   dfCustSup.to_csv('3-CustomerSupport.csv',index=False)
   print("Step#2 Added Customer Support Successfully")

print("=====================================================")

In [ ]:
dfDC=pd.DataFrame(columns=['datacenter_name','address','company_id','customer_id'])
mapCols=[ ('datacenter_name','company_name'),
         ('address','company_address'),
         ('company_id','company_id') ,
         ('customer_id','id')
        ]

for colPair in mapCols:
    dfDC[colPair[0]]=dfCustSup[colPair[1]]   
print(dfDC)

dfDC['updated_at']=now_dt

print("=====================================================")
if len(dfDC)==len(dfCustSup):
    
   result=add_data_values(dfDC,'app_datacenter',get_postgres_conn())
   sql_subcomp="""
select dc.id,dc.datacenter_name,ac.customer_name,dc.customer_id,
dc.company_id as compnay_id_dc,
ac.company_id as comp_id_cust from app_datacenter dc

inner join app_customer ac on ac.id = dc.customer_id
where dc.company_id  in  %(id_param)s  order by id
   """
   dfDC=list_data(sql_subcomp,{"id_param":tuple(lisCompID)},get_postgres_conn()) 
   print(dfDC)
   dfDC.to_csv('4-DataCenter.csv',index=False)
   print("Step#3 Added DataCenter Successfully")
    
    
    
    

In [ ]:
dfBranch=pd.DataFrame(columns=['branch_name','address','company_id','customer_id'])
mapCols=[ ('branch_name','company_name'),
         ('address','company_address'),
         ('company_id','company_id') ,
         ('customer_id','id')
        ]

for colPair in mapCols:
    dfBranch[colPair[0]]=dfCustSup[colPair[1]]   
print(dfBranch)

dfBranch['updated_at']=now_dt


print("=====================================================")
if len(dfBranch)==len(dfCustSup):
    
   result=add_data_values(dfBranch,'app_branch',get_postgres_conn())
   sql_subcomp="""
select branch.id,branch.branch_name,ac.customer_name,branch.customer_id,
branch.company_id as compnay_id_dc,
ac.company_id as comp_id_cust from app_branch branch

inner join app_customer ac on ac.id = branch.customer_id
where branch.company_id  in  %(id_param)s  order by id
   """
   dfBranch=list_data(sql_subcomp,{"id_param":tuple(lisCompID)},get_postgres_conn()) 
   print(dfBranch)
   dfBranch.to_csv('5-Branch.csv',index=False)
   print("Added Branch Successfully")
    